In [54]:
import requests
import re
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import datetime
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import warnings
# warnings.filterwarnings('ignore', 
#                        message='The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.')
pd.set_option('display.max_columns', None)

In [55]:
from webdriver_manager.chrome import ChromeDriverManager
options = webdriver.ChromeOptions()
options.add_argument("--verbose")
options.add_argument('--no-sandbox')
options.add_argument("--window-size=1920, 1200")
# options.add_argument('--headless')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")
svc = webdriver.ChromeService(executable_path="C:\\Users\\mfeld\\Desktop\\New folder\\chromedriver.exe")

In [ ]:
def convert(string):
    datetime_str = np.datetime64(datetime.datetime.strptime(string, '%b %Y').strftime('%Y-%m'))
    return datetime_str

In [56]:
df = pd.DataFrame({
    "ID": pd.Series(dtype="str"), "Link": pd.Series(dtype="str"), "Title": pd.Series(dtype="str"), "PostingDate": pd.Series(dtype="str"),
    "Location": pd.Series(dtype="str"), "AdType": pd.Series(dtype="str"), "Price": pd.Series(dtype="Int64"),
    "RentalFrequency": pd.Series(dtype="str"), "Area": pd.Series(dtype="Int64"), "SpaceType": pd.Series(dtype="str"), "Furnished": pd.Series(dtype="bool"),
    "Insurance": pd.Series(dtype="Int64"), "Deposit": pd.Series(dtype="Int64"), "PriceType": pd.Series(dtype="str"),
    "AmenitiesList": pd.Series(dtype="object"),
    "Description": pd.Series(dtype="str"),
    "Publisher": pd.Series(dtype="str"), "PublisherPhone": pd.Series(dtype="str"), 
    "PublisherProfileLink": pd.Series(dtype="str"), "PublisherJoiningDate": pd.Series(dtype="str")})

In [19]:
list_page = requests.get("https://www.dubizzle.com.eg/en/properties/commercial-for-rent/new-cairo/")
sleep(5)
soup = BeautifulSoup(list_page.content)
listings = soup.find_all('li', attrs={"aria-label":"Listing"})
print(f"Number of listings: {len(listings)}")
ads = []
for i in range(len(listings)):
    ads.append(listings[i].find('div', class_="_70cdfb32"))
print(f"Number of ads: {len(ads)}")

driver = webdriver.Chrome(options=options, service=svc)

for i in range(len(ads)):
    ad_link = "https://www.dubizzle.com.eg" + ads[i].find('a')['href']
    print(f"{i}: {ad_link}")

    content = requests.get(ad_link).content
    sleep(5)
    ad_page = BeautifulSoup(content)

    Title=PostingDate=Location=AdType=Price=RentalFrequency=Area=SpaceType=Furnished = None
    Insurance=Deposit=AmenitiesList=Description=Publisher=PublisherPhone=PublisherProfileLink=PublisherJoiningDate = None
    
    try:
        ID = str(ad_page.find('div', class_="_948d9e0a _84a14222 d7383df5 _371e9918").find('div', class_="_114d5a00").text)[6:]
    except:
        ID = None

    Title = str(ad_page.find('h1', class_="_75bce902").text)
    PostingDate = str(ad_page.find('span', attrs={"aria-label":"Creation date"}).text)
    Location = str(ad_page.find('span', attrs={"aria-label": "Location", "class": "_8206696c"}).text)
    AdType = str(ad_page.find('span', class_="_1fcb6673 b7af14b4").text)
    Price = int(str(ad_page.find('span', attrs={"aria-label": "Price", "class": "_24469da7"}).text)[4:].replace(",", ""))
    
    Highlights = ad_page.find_all('div', class_="_948d9e0a e655db77 _371e9918 _95d4067f")
    for div in Highlights:
        label = str(div.find('span', class_="_8206696c").text)
        value = str(div.find('span', class_="_8206696c b7af14b4").text)
    
        if label == "Type":
            SpaceType = value
        elif label == "Rental Frequency":
            RentalFrequency = value
        elif "Area (" in label:
            Area = int(value.replace(",", ""))
        elif label == "Furnished":
            Furnished = True if value == "Yes" else False
    
    Details = ad_page.find_all('div', class_="_9a8eacd9")
    for div in Details:
        spans = div.find_all("span")
        if spans[0].text == "Insurance":
            Insurance = int(spans[1].text.replace(",", ""))
        elif spans[0].text == "Deposit":
            Deposit = int(spans[1].text.replace(",", ""))
    
    Amenities = ad_page.find('div', attrs={"aria-label":"Features", "class":"_2961c394"})
    spans = Amenities.find_all("span", class_="c327b807")
    temp = []
    for feature in spans:
        temp.append(str(feature.text))
    if temp:
        AmenitiesList = temp
    else:
        AmenitiesList = None
    
    DescriptionTag = ad_page.find('div', attrs={"aria-label":"Description", "class":"_2961c394"})
    div = DescriptionTag.find("div", class_="_472bfbef")
    Description = re.sub('\n', '', str(div.get_text()))



    SellerSection = ad_page.find('div', attrs={'class': '_2961c394', 'aria-label': 'Seller description'})

    PublisherProfileLink = "https://www.dubizzle.com.eg" + SellerSection.find('a', href=True)['href']
    Publisher = SellerSection.find('span', class_='_8206696c b7af14b4').get_text(strip=True)
    PublisherJoiningDate = SellerSection.find('div', class_='_7117d2a4').find('span', class_='_8206696c').get_text(strip=True)

    driver.get(ad_link)
    try:
        SellerSection = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div[aria-label="Seller description"]'))
        )

        PhoneButton = SellerSection.find_element(By.CSS_SELECTOR, 'button._95e95b19.e07f63ca[type="submit"]')

        PhoneButton = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button._95e95b19.e07f63ca[type="submit"]'))
        )
        PhoneButton.click()

        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'span._76bf0655._1edc4691.c5efc674.b7af14b4'))
        )

        PhoneNumberElement = driver.find_element(By.CSS_SELECTOR, 'span._76bf0655._1edc4691.c5efc674.b7af14b4')
        PublisherPhone = PhoneNumberElement.text
        # print(PublisherPhone)

    except Exception as e:
        print("Publisher phone error", e)
    
    DataPoint = {
    "ID": ID, "Link": ad_link, "Title": Title, "PostingDate": PostingDate,
    "Location": Location, "AdType": AdType, "Price": Price,
    "RentalFrequency": RentalFrequency, "Area": Area, "SpaceType": SpaceType, "Furnished": Furnished,
    "Insurance": Insurance, "Deposit": Deposit, "AmenitiesList": AmenitiesList,
    "Description": Description,
    "Publisher": Publisher, "PublisherPhone": PublisherPhone, 
    "PublisherProfileLink": PublisherProfileLink, "PublisherJoiningDate": PublisherJoiningDate }
    df = pd.concat([df, pd.DataFrame([DataPoint])], ignore_index=True)

    if i % 10 == 0:
        df.to_csv("rentalOffices.csv", mode='a', header=not os.path.exists("rentalOffices.csv"))

Number of listings: 45
Number of ads: 45
0: https://www.dubizzle.com.eg/en/ad/office-for-sale-or-rent-in-the-north-90th-street-business-plus-building-ID501315708.html
1: https://www.dubizzle.com.eg/en/ad/open-plan-office-space-for-10-persons-in-cairo-new-cairo-ID500687927.html
2: https://www.dubizzle.com.eg/en/ad/clinic-for-rent-41m-second-floor-mivida-new-cairo-ID501346838.html
3: https://www.dubizzle.com.eg/en/ad/restaurant-cafe-for-rent-ID501445983.html
4: https://www.dubizzle.com.eg/en/ad/office-for-rent-fully-finished-administrative-office-in-new-cairo-90th-street-ID501410463.html
5: https://www.dubizzle.com.eg/en/ad/administrative-office-for-rent-in-al-rehab-50-square-meters-fully-equipped-ID501450276.html
6: https://www.dubizzle.com.eg/en/ad/%D8%B9%D9%8A%D8%A7%D8%AF%D9%87-%D9%84%D9%84%D8%A7%D9%8A%D8%AC%D8%A7%D8%B1-%D9%81%D9%8A-%D8%A7%D9%84%D8%AA%D8%AC%D9%85%D8%B9-%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3-%D9%81%D9%8A-%D8%A7%D9%84%D9%85%D8%B3%D8%AA%D8%AB%D9%85%D8%B1%D9%8A%D9%86-%D8%A7

In [57]:
df.head()

,ID,Link,Title,PostingDate,Location,AdType,Price,RentalFrequency,Area,SpaceType,Furnished,Insurance,Deposit,PriceType,AmenitiesList,Description,Publisher,PublisherPhone,PublisherProfileLink,PublisherJoiningDate


In [60]:
base = "https://www.dubizzle.com.eg/en/properties/commercial-for-rent/new-cairo/"
# base = "https://www.dubizzle.com.eg/en/properties/commercial-for-rent/new-cairo/?page=2"
page = 74
driver = webdriver.Chrome(options=options, service=svc)
while True:
    page += 1
    if page == 1:
        url = base
    else:
        url = base + "?page=" + str(page)
    print(f"page{page}: {url}")
    list_page = requests.get(url)
    sleep(5)
    if list_page.status_code != 200:
        print(f"\n\n{url} not found")
        print(list_page)
        break
    sleep(5)
    soup = BeautifulSoup(list_page.content)
    listings = soup.find_all('li', attrs={"aria-label":"Listing"})
    while not listings:
        print("Retrying...")
        list_page = requests.get(url)
        sleep(5)
        soup = BeautifulSoup(list_page.content)
        listings = soup.find_all('li', attrs={"aria-label":"Listing"})
    print(f"Number of listings: {len(listings)}")
    ads = []
    for i in range(len(listings)):
        ads.append(listings[i].find('div', class_="_70cdfb32"))
    print(f"Number of ads: {len(ads)}")
    
    for i in range(len(ads)):
        ad_link = "https://www.dubizzle.com.eg" + ads[i].find('a')['href']
        print(f"{i}: {ad_link}")

        content = requests.get(ad_link).content
        sleep(5)
        ad_page = BeautifulSoup(content)
    
        Title=PostingDate=Location=AdType=Price=RentalFrequency=Area=SpaceType=Furnished = None
        Insurance=Deposit=PriceType=AmenitiesList=Description=Publisher=PublisherPhone=PublisherProfileLink=PublisherJoiningDate = None

        try:
            ID = str(ad_page.find('div', class_="_948d9e0a _84a14222 d7383df5 _371e9918").find('div', class_="_114d5a00").text)[6:]
        except:
            ID = None

        Overview = ad_page.find('div', attrs={"class":"_2961c394", "aria-label":"Overview"})
        try:
            Title = str(Overview.find('h1', class_="_75bce902").text)
        except:
            Title = None
        
        Dates = ad_page.find_all('span', class_="_8206696c")
        for date in Dates:
            if date.find('span', attrs={"aria-label":"Creation date"}) is not None:
                PostingDate = str(ad_page.find('span', attrs={"aria-label":"Creation date"}).text)

        Location = str(ad_page.find('span', attrs={"aria-label": "Location", "class": "_8206696c"}).text)
        try:
            AdType = str(ad_page.find('span', class_="_1fcb6673 b7af14b4").text)
        except:
            AdType = None
        Price = int(str(ad_page.find('span', attrs={"aria-label": "Price", "class": "_24469da7"}).text)[4:].replace(",", ""))
        
        Highlights = ad_page.find_all('div', class_="_948d9e0a e655db77 _371e9918 _95d4067f")
        for div in Highlights:
            label = str(div.find('span', class_="_8206696c").text)
            value = str(div.find('span', class_="_8206696c b7af14b4").text)
        
            if label == "Type":
                SpaceType = value
            elif label == "Rental Frequency":
                RentalFrequency = value
            elif "Area (" in label:
                Area = value.replace(",", "")
                Area = Area.split(".")[0]
                Area = int(Area)
            elif label == "Furnished":
                Furnished = True if value == "Yes" else False
        
        Details = ad_page.find('div', attrs={"class":"_2961c394", "aria-label":"Details"})
        if Details is None:
            Insurance = Deposit = PriceType = None
        else:
            Details = Details.find_all('div', class_="_9a8eacd9")
            for div in Details:
                spans = div.find_all("span")
                if spans[0].text == "Insurance":
                    Insurance = int(spans[1].text.split(".")[0].replace(",", ""))
                elif spans[0].text == "Deposit":
                    Deposit = int(spans[1].text.split(".")[0].replace(",", ""))
                elif spans[0].text == "Price Type":
                    PriceType = str(spans[1].text)
        
        Amenities = ad_page.find('div', attrs={"aria-label":"Features", "class":"_2961c394"})
        spans = Amenities.find_all("span", class_="c327b807")
        temp = []
        for feature in spans:
            temp.append(str(feature.text))
        if temp:
            AmenitiesList = temp
        else:
            AmenitiesList = None
        
        DescriptionTag = ad_page.find('div', attrs={"aria-label":"Description", "class":"_2961c394"})
        div = DescriptionTag.find("div", class_="_472bfbef")
        Description = re.sub('\n', '', str(div.get_text()))
    
    
    
        SellerSection = ad_page.find('div', attrs={'class': '_2961c394', 'aria-label': 'Seller description'})
    
        PublisherProfileLink = "https://www.dubizzle.com.eg" + SellerSection.find('a', href=True)['href']
        Publisher = SellerSection.find('span', class_='_8206696c b7af14b4').get_text(strip=True)
        PublisherJoiningDate = SellerSection.find('div', class_='_7117d2a4').find('span', class_='_8206696c').get_text(strip=True)
    
        driver.get(ad_link)
        try:
            SellerSection = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[aria-label="Seller description"]'))
            )
    
            PhoneButton = SellerSection.find_element(By.CSS_SELECTOR, 'button._95e95b19.e07f63ca[type="submit"]')
    
            PhoneButton = WebDriverWait(driver, 30).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'button._95e95b19.e07f63ca[type="submit"]'))
            )
            PhoneButton.click()
    
            WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'span._76bf0655._1edc4691.c5efc674.b7af14b4'))
            )
    
            PhoneNumberElement = driver.find_element(By.CSS_SELECTOR, 'span._76bf0655._1edc4691.c5efc674.b7af14b4')
            PublisherPhone = PhoneNumberElement.text
            # print(PublisherPhone)
    
        except Exception as e:
            print("Publisher phone error", e)
        
        DataPoint = {
        "ID": ID, "Link": ad_link, "Title": Title, "PostingDate": PostingDate,
        "Location": Location, "AdType": AdType, "Price": Price,
        "RentalFrequency": RentalFrequency, "Area": Area, "SpaceType": SpaceType, "Furnished": Furnished,
        "Insurance": Insurance, "Deposit": Deposit, "PriceType": PriceType,
        "AmenitiesList": AmenitiesList,
        "Description": Description,
        "Publisher": Publisher, "PublisherPhone": PublisherPhone, 
        "PublisherProfileLink": PublisherProfileLink, "PublisherJoiningDate": PublisherJoiningDate }
        df = pd.concat([df, pd.DataFrame([DataPoint])], ignore_index=True)
    
        if i % 10 == 0:
            df.to_csv("rentalOffices.csv", mode='a', header=not os.path.exists("rentalOffices.csv"))
    

page75: https://www.dubizzle.com.eg/en/properties/commercial-for-rent/new-cairo/?page=75
Number of listings: 45
Number of ads: 45
0: https://www.dubizzle.com.eg/en/ad/%D9%85%D8%AD%D9%84-160%D9%85-%D9%84%D9%88%D8%B1%D8%A7%D9%86-%D8%B9%D9%84%D9%8A-%D8%A7%D9%84%D8%A8%D8%AD%D8%B1-%D9%85%D8%A8%D8%A7%D8%B4%D8%B1%D8%A9-ID203562739.html
1: https://www.dubizzle.com.eg/en/ad/%D9%85%D8%AD%D9%84-%D9%84%D9%84%D8%A7%D9%8A%D8%AD%D8%A7%D8%B1-%D8%B4%D8%A7%D8%B1%D8%B9-%D8%A7%D9%84%D8%AA%D8%B1%D8%A7%D9%85-%D8%A7%D9%84%D8%B1%D8%A1%D9%8A%D8%B3%D9%8A-%D8%A8%D8%A7%D9%84%D9%82%D8%B1%D8%A8-%D9%85%D9%86-%D8%B4%D8%A7%D8%B1%D8%B9-%D8%A7%D9%84%D8%AC%D9%88%D8%A7%D9%87%D8%B1-ID203594627.html
Publisher phone error Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF61BE2FBE5+28741]
	(No symbol) [0x00007FF61BD97890]
	(No symbol) [0x00007FF61BC34FDA]
	(No symbol) [0x00007FF61BC89322]
	(No symbol) [0x00007FF61BC8954C]
	(No symbol) [0x00007FF61BCD3337]
	(No symbol) [0x00007FF61BCAFF4F]
	(No symbol) [0x00007FF61BCD00B4]
	(